In [1]:
import os
import struct
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt
%matplotlib inline

d:\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 . Create Dataset -- add random noise pics into MNIST

In [2]:
def load_data(path, kind=''):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [3]:
path = './datasets/'

In [4]:
num_classes = 10 

In [5]:
X_train,y_train = load_data(path, kind='train')
X_test,y_test = load_data(path, kind='t10k')

In [6]:
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

X_sparse_train = coo_matrix(X_train)
X_train, X_sparse_train, y_train = shuffle(X_train, X_sparse_train, y_train, random_state=42)

In [7]:
def keras_style_ds(X_train, X_test, y_train, y_test):
# keras - input
    X_train = X_train / 255
    X_test  = X_test / 255

    X_train = X_train.reshape(len(X_train),1,28,28).astype('float64')
    X_test = X_test.reshape(len(X_test),1,28,28).astype('float64')

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    
    return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = keras_style_ds(X_train, X_test, y_train, y_test)

# 2 . Build Model -- keras model

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

K.clear_session()

In [10]:
def model_factory():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', 
                     data_format='channels_first',
                     input_shape=(1, 28, 28)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
    
    return model


In [11]:
# model = model_factory()
# model.summary()

# 3. Training

In [12]:
hs_test_scores = []
hs_history=[]
for i in range(30):
    model = model_factory()
    
    
    history = model.fit(X_train, y_train, epochs=12, batch_size=128,validation_split=0.1,verbose=1)
    hs_history.append(history)
    model.save("./models/MNIST_0418_nofold_nonoise_%d.h5" % (i+1))

    print("------------------------%d : test ------------------------------------------" % (i+1))
    test_score = model.evaluate(X_test, y_test, verbose=1)
    hs_test_scores.append(test_score)
    print("%d : Mnist test sets -> Loss: %.2f%%" % (i+1, test_score[0]))
    print("%d : Mnist test sets -> Accuracy: %.2f%%" % (i+1, test_score[1]*100))
    print()

Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 15s 279us/step - loss: 0.5191 - acc: 0.8335 - val_loss: 0.0999 - val_acc: 0.9682
Epoch 2/12
54000/54000 [==============================] - 12s 227us/step - loss: 0.1457 - acc: 0.9593 - val_loss: 0.0699 - val_acc: 0.9780
Epoch 3/12
54000/54000 [==============================] - 12s 228us/step - loss: 0.1026 - acc: 0.9718 - val_loss: 0.0564 - val_acc: 0.9822
Epoch 4/12
54000/54000 [==============================] - 12s 227us/step - loss: 0.0844 - acc: 0.9761 - val_loss: 0.0493 - val_acc: 0.9837
Epoch 5/12
54000/54000 [==============================] - 12s 228us/step - loss: 0.0749 - acc: 0.9796 - val_loss: 0.0433 - val_acc: 0.9860
Epoch 6/12
54000/54000 [==============================] - 12s 227us/step - loss: 0.0644 - acc: 0.9825 - val_loss: 0.0515 - val_acc: 0.9860
Epoch 7/12
54000/54000 [==============================] - 12s 228us/step - loss: 0.0583 - acc: 0.9837 - val_loss: 0.0

Epoch 4/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0866 - acc: 0.9758 - val_loss: 0.0424 - val_acc: 0.9853
Epoch 5/12
54000/54000 [==============================] - 12s 229us/step - loss: 0.0722 - acc: 0.9804 - val_loss: 0.0528 - val_acc: 0.9840
Epoch 6/12
54000/54000 [==============================] - 12s 230us/step - loss: 0.0640 - acc: 0.9819 - val_loss: 0.0390 - val_acc: 0.9872
Epoch 7/12
54000/54000 [==============================] - 12s 229us/step - loss: 0.0562 - acc: 0.9850 - val_loss: 0.0402 - val_acc: 0.9878
Epoch 8/12
54000/54000 [==============================] - 12s 229us/step - loss: 0.0515 - acc: 0.9860 - val_loss: 0.0348 - val_acc: 0.9885
Epoch 9/12
54000/54000 [==============================] - 13s 232us/step - loss: 0.0485 - acc: 0.9868 - val_loss: 0.0336 - val_acc: 0.9908
Epoch 10/12
54000/54000 [==============================] - 12s 229us/step - loss: 0.0436 - acc: 0.9879 - val_loss: 0.0346 - val_acc: 0.9888
Epoch 11/12
54000/54000 [=

54000/54000 [==============================] - 12s 229us/step - loss: 0.0595 - acc: 0.9835 - val_loss: 0.0479 - val_acc: 0.9847
Epoch 8/12
54000/54000 [==============================] - 12s 230us/step - loss: 0.0518 - acc: 0.9859 - val_loss: 0.0401 - val_acc: 0.9900
Epoch 9/12
54000/54000 [==============================] - 12s 229us/step - loss: 0.0475 - acc: 0.9865 - val_loss: 0.0373 - val_acc: 0.9883
Epoch 10/12
54000/54000 [==============================] - 12s 228us/step - loss: 0.0436 - acc: 0.9874 - val_loss: 0.0412 - val_acc: 0.9903
Epoch 11/12
54000/54000 [==============================] - 12s 229us/step - loss: 0.0430 - acc: 0.9888 - val_loss: 0.0360 - val_acc: 0.9905
Epoch 12/12
54000/54000 [==============================] - 12s 228us/step - loss: 0.0399 - acc: 0.9888 - val_loss: 0.0400 - val_acc: 0.9895
------------------------9 : test ------------------------------------------
10000/10000 [==============================] - 1s 140us/step
9 : Mnist test sets -> Loss: 0.03%
9 

54000/54000 [==============================] - 12s 230us/step - loss: 0.0436 - acc: 0.9878 - val_loss: 0.0430 - val_acc: 0.9895
Epoch 11/12
54000/54000 [==============================] - 12s 230us/step - loss: 0.0418 - acc: 0.9884 - val_loss: 0.0376 - val_acc: 0.9895
Epoch 12/12
54000/54000 [==============================] - 12s 230us/step - loss: 0.0377 - acc: 0.9892 - val_loss: 0.0372 - val_acc: 0.9898
------------------------13 : test ------------------------------------------
10000/10000 [==============================] - 1s 129us/step
13 : Mnist test sets -> Loss: 0.03%
13 : Mnist test sets -> Accuracy: 99.18%

Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 14s 255us/step - loss: 0.5031 - acc: 0.8389 - val_loss: 0.1024 - val_acc: 0.9695
Epoch 2/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.1509 - acc: 0.9580 - val_loss: 0.0655 - val_acc: 0.9795
Epoch 3/12
54000/54000 [=========================

10000/10000 [==============================] - 1s 132us/step
17 : Mnist test sets -> Loss: 0.03%
17 : Mnist test sets -> Accuracy: 99.14%

Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 14s 261us/step - loss: 0.5023 - acc: 0.8418 - val_loss: 0.0993 - val_acc: 0.9690
Epoch 2/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.1463 - acc: 0.9595 - val_loss: 0.0691 - val_acc: 0.9778
Epoch 3/12
54000/54000 [==============================] - 13s 233us/step - loss: 0.1068 - acc: 0.9716 - val_loss: 0.0603 - val_acc: 0.9828
Epoch 4/12
54000/54000 [==============================] - 13s 232us/step - loss: 0.0888 - acc: 0.9757 - val_loss: 0.0533 - val_acc: 0.9840
Epoch 5/12
54000/54000 [==============================] - 13s 232us/step - loss: 0.0740 - acc: 0.9799 - val_loss: 0.0491 - val_acc: 0.9848
Epoch 6/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0678 - acc: 0.9815 - val_loss: 0.0

54000/54000 [==============================] - 13s 233us/step - loss: 0.1491 - acc: 0.9593 - val_loss: 0.0729 - val_acc: 0.9765
Epoch 3/12
54000/54000 [==============================] - 13s 232us/step - loss: 0.1078 - acc: 0.9701 - val_loss: 0.0525 - val_acc: 0.9822
Epoch 4/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0878 - acc: 0.9754 - val_loss: 0.0550 - val_acc: 0.9837
Epoch 5/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0734 - acc: 0.9791 - val_loss: 0.0453 - val_acc: 0.9872
Epoch 6/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0648 - acc: 0.9825 - val_loss: 0.0471 - val_acc: 0.9883
Epoch 7/12
54000/54000 [==============================] - 13s 233us/step - loss: 0.0593 - acc: 0.9838 - val_loss: 0.0407 - val_acc: 0.9887
Epoch 8/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0528 - acc: 0.9856 - val_loss: 0.0386 - val_acc: 0.9898
Epoch 9/12
54000/54000 [==============

54000/54000 [==============================] - 13s 232us/step - loss: 0.0677 - acc: 0.9812 - val_loss: 0.0416 - val_acc: 0.9875
Epoch 6/12
54000/54000 [==============================] - 12s 231us/step - loss: 0.0588 - acc: 0.9840 - val_loss: 0.0360 - val_acc: 0.9887
Epoch 7/12
54000/54000 [==============================] - 13s 237us/step - loss: 0.0536 - acc: 0.9856 - val_loss: 0.0374 - val_acc: 0.9890
Epoch 8/12
54000/54000 [==============================] - 13s 236us/step - loss: 0.0488 - acc: 0.9859 - val_loss: 0.0412 - val_acc: 0.9888
Epoch 9/12
54000/54000 [==============================] - 13s 234us/step - loss: 0.0440 - acc: 0.9878 - val_loss: 0.0494 - val_acc: 0.9865
Epoch 10/12
54000/54000 [==============================] - 13s 235us/step - loss: 0.0419 - acc: 0.9883 - val_loss: 0.0396 - val_acc: 0.9890
Epoch 11/12
54000/54000 [==============================] - 13s 234us/step - loss: 0.0388 - acc: 0.9894 - val_loss: 0.0447 - val_acc: 0.9890
Epoch 12/12
54000/54000 [===========

54000/54000 [==============================] - 13s 237us/step - loss: 0.0546 - acc: 0.9851 - val_loss: 0.0412 - val_acc: 0.9873
Epoch 9/12
54000/54000 [==============================] - 13s 235us/step - loss: 0.0490 - acc: 0.9866 - val_loss: 0.0445 - val_acc: 0.9880
Epoch 10/12
54000/54000 [==============================] - 13s 235us/step - loss: 0.0469 - acc: 0.9874 - val_loss: 0.0432 - val_acc: 0.9878
Epoch 11/12
54000/54000 [==============================] - 13s 236us/step - loss: 0.0424 - acc: 0.9882 - val_loss: 0.0428 - val_acc: 0.9883
Epoch 12/12
54000/54000 [==============================] - 13s 237us/step - loss: 0.0400 - acc: 0.9894 - val_loss: 0.0409 - val_acc: 0.9893
------------------------30 : test ------------------------------------------
10000/10000 [==============================] - 2s 174us/step
30 : Mnist test sets -> Loss: 0.04%
30 : Mnist test sets -> Accuracy: 99.00%



In [13]:
hs_test_scores

[[0.03319059042234185, 0.9909],
 [0.03507601902729175, 0.99],
 [0.030431764418181045, 0.9916],
 [0.03132552981669423, 0.9923],
 [0.036338698807626496, 0.9904],
 [0.02747381137215634, 0.9917],
 [0.027489089398914077, 0.9921],
 [0.03362245638066015, 0.9907],
 [0.03326312834405435, 0.9904],
 [0.03778266454554951, 0.9899],
 [0.025910327440304717, 0.9918],
 [0.03637157477012879, 0.9893],
 [0.027183021672891845, 0.9918],
 [0.030699622026029647, 0.9911],
 [0.03130165968582078, 0.9914],
 [0.027543761366804028, 0.992],
 [0.032645623915310173, 0.9914],
 [0.02886422800100663, 0.9915],
 [0.03448929514521715, 0.9915],
 [0.03478842351702415, 0.9905],
 [0.029562193160516472, 0.9913],
 [0.031689561881652846, 0.992],
 [0.03476876900211473, 0.991],
 [0.03249616706682045, 0.9903],
 [0.030067334942458092, 0.9925],
 [0.02740582233781024, 0.9918],
 [0.03199739393116506, 0.9917],
 [0.03020890845848244, 0.9911],
 [0.029885735045072217, 0.9914],
 [0.040910371956364086, 0.99]]